In [2]:
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Input, Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.layers.merge import concatenate
from image_segmentation_keras.keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob

import sys
sys.path.insert(1, './src')
from crfrnn_layer import CrfRnnLayer

Using TensorFlow backend.


In [9]:
import keras
import tensorflow
from tensorflow import keras as k
print(keras.__version__, tensorflow.__version__, k.__version__)

2.3.1 2.2.0 2.3.0-tf


In [2]:
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPooling2D((2, 2))(x)
        p = Dropout(0.15)(p)
        return x, p
    else:
        return x
def build_unet(shape, num_classes):
    img_input = Input(shape)
    o = Dropout(0.05)(img_input)

    """ Encoder """
    x1, p1 = conv_block(o, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = concatenate([u1, x4],axis=3)
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = concatenate([u2, x3],axis=3)
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = concatenate([u3, x2],axis=3)
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = concatenate([u4, x1],axis=3)
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="relu")(x8)
    
    crf_output = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=num_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([output, img_input])

    return get_segmentation_model(img_input ,  crf_output )


In [3]:
shape = (256, 256, 3)
num_classes = 3
input_height=256 
input_width=256
model = build_unet(shape, num_classes)


In [4]:
#shuffled
model.train(
    train_images =  "/Users/mavaylon/Research/Data1/train/img/",
    train_annotations = "/Users/mavaylon/Research/Data1/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/train/img/*")),
    batch_size=1,
    validate=True,
    val_images="/Users/mavaylon/Research/Data1/test/img/",
    val_annotations="/Users/mavaylon/Research/Data1/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/test/img/*"))
)

Verifying training dataset


  0%|          | 0/1478 [00:00<?, ?it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:07<00:00, 193.84it/s]


Dataset verified! 
correct
Epoch 1/20
  41/5912 [..............................] - ETA: 4:43:22 - loss: 0.8715 - accuracy: 0.6231

KeyboardInterrupt: 

In [ ]:
!pip list

In [17]:
model.load_weights("/Users/mavaylon/Research/LBNL_Segmentation_crf/unet_pet_class.h5")

In [ ]:
model.trai